## For each bee id, be able to provide the number of detections in the hive in the given timeslot.

In [25]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import numpy as np
import pandas as pd
import psycopg2
import psycopg2.extras
from datetime import timedelta, datetime

import bb_utils
import bb_utils.meta
import bb_utils.ids
import bb_backend
from bb_backend.api import FramePlotter, VideoPlotter
from bb_backend.api import get_plot_coordinates, transform_axis_coordinates, get_image_origin

bb_backend.api.server_adress = 'localhost:8000'
m = bb_utils.meta.BeeMetaInfo()
connect_str = """dbname='beesbook' user='reader' host='tonic.imp.fu-berlin.de' 
                 password='' application_name='mehmed'"""

In [26]:
datetime_start = datetime(2016, 8, 23) #First detections are on 20.07.2016, last are 19.09.2016 (3 months duration)
datetime_end = datetime_start + timedelta(hours=1)

In [27]:
with psycopg2.connect(connect_str) as conn:
    query = """SELECT * FROM bb_detections
               WHERE timestamp >= %s AND
                     timestamp < %s AND
                     bee_id_confidence >= %s
               ;"""
    df = pd.read_sql_query(
        query, conn, 
        params=(datetime_start, datetime_end, 0.99), #note: high confidence requirement
        coerce_float=False)

In [28]:
meta = bb_utils.meta.BeeMetaInfo()


group_idx = 20
group_ids = map(lambda i: i.as_ferwar(), 
                list(map(bb_utils.ids.BeesbookID.from_dec_12, meta.get_foragergroup(group_idx).dec12)))

group_ids = list(group_ids)

group = meta.get_foragergroup(group_idx)
print(group.date)

2016-08-23 00:00:00


In [29]:
print(df.shape)
single_bee_df = df.loc[df['bee_id'] == 282]
print(single_bee_df.shape)
df.head()

(3409204, 13)
(0, 13)


,timestamp,frame_id,detection_idx,track_id,x_pos,y_pos,orientation,x_pos_hive,y_pos_hive,orientation_hive,bee_id,bee_id_confidence,cam_id
0,2016-08-23 00:27:31.780472+00:00,11155435335597279427,52,10246467627072216286,3025,2439,-0.023102,NaN,NaN,NaN,3073,1.0,0
1,2016-08-23 00:27:32.116578+00:00,15196065758002572934,52,10246467627072216286,3051,2426,-0.254920,NaN,NaN,NaN,3073,1.0,0
2,2016-08-23 00:27:32.447851+00:00,16990487867506213711,51,10246467627072216286,3062,2414,-0.216924,NaN,NaN,NaN,3073,1.0,0
3,2016-08-23 00:27:32.783652+00:00,17014105345143278317,50,10246467627072216286,3075,2401,-0.281169,NaN,NaN,NaN,3073,1.0,0
4,2016-08-23 00:27:33.109483+00:00,15362768355940034791,50,10246467627072216286,3099,2389,-0.403018,NaN,NaN,NaN,3073,1.0,0


In [30]:
num_intervals = 10
interval_length = (datetime_end - datetime_start) // 5

#create zeros in the shape required by (bees x num_invervals)
intervals = pd.DataFrame(data=np.zeros([len(group_ids),5])) 
ids = pd.DataFrame(data={'id': group_ids})
ids = pd.concat([ids, intervals], axis=1)

In [31]:
interval_starttime = datetime_start
for i in range(num_intervals): 
    #check detections
    interval_endtime = interval_starttime + interval_length
    before = df['timestamp'] >= interval_starttime 
    after = df['timestamp'] < interval_endtime
    interval_detections = df[before & after]
    print(interval_detections.shape)
    n = 0
    for b in ids['id']:
        if b in interval_detections['bee_id'].unique():
            ids.set_value(n, i, 1)
        n += 1 
    interval_starttime = interval_endtime

(680073, 13)
(671556, 13)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if sys.path[0] == '':


(677343, 13)
(668086, 13)
(712146, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)


In [32]:
print(ids.shape, "\nBeginning of observation ", datetime_start, "\nObservation period is 10min, divided into 5 2-minute timeslots.")
ids

(11, 6) 
Beginning of observation  2016-08-23 00:00:00 
Observation period is 10min, divided into 5 2-minute timeslots.


,id,0,1,2,3,4
0,1799,0.0,0.0,0.0,0.0,0.0
1,1593,0.0,1.0,1.0,1.0,1.0
2,2106,1.0,0.0,0.0,0.0,0.0
3,1662,1.0,0.0,0.0,0.0,0.0
4,1180,0.0,0.0,0.0,1.0,1.0
5,2984,0.0,0.0,0.0,0.0,1.0
6,1197,0.0,0.0,0.0,0.0,0.0
7,1714,0.0,0.0,0.0,0.0,0.0
8,1471,1.0,1.0,0.0,1.0,0.0
9,1232,0.0,0.0,0.0,0.0,0.0


In [33]:
# df['bee_id'].value_counts()

In [34]:
#we have data from 10 minutes / 2 hours / 48 hours
